In [1]:
import os
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from utils import loadWord2Vec, clean_str
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from scipy.spatial.distance import cosine


In [2]:
line_numbers = [110224, 146930, 2941, 104545, 62327, 29760, 96891, 47026, 117733, 163207, 61451, 20590, 145423, 33883, 4524, 81255, 82144, 85139, 167094, 125904, 116418, 158098, 95233, 81836, 84071, 53850, 112264, 17238, 34056, 10708, 164819, 32413, 75065, 139894, 161233, 100688, 148030, 55853, 23077, 152167, 44378, 111683, 145091, 132167, 83409, 153865, 143858, 111283, 150657, 36240, 142480, 112754, 149469, 50328, 163801, 45304, 155756, 151190, 160715, 8876, 116827, 142384, 143574, 136796, 278, 120989, 120991, 151642, 44083, 94917, 32815, 117614, 98217, 41937, 16626, 163885, 69261, 29037, 91370, 31452, 161506, 27949, 29236, 15855, 162541, 161900, 157800, 90014, 120276, 57310, 92875, 140990, 11693, 78368, 62286, 80503, 72673, 121791, 12663, 90678]

In [3]:
node_ids=[]
for line_no in line_numbers:
    node_id = line_no - 1
    node_ids.append(node_id)

node_ids.sort()

In [4]:
print(node_ids[:5])

[277, 2940, 4523, 8875, 10707]


In [5]:

word_embeddings_dim = 300
word_vector_map = {}

# shulffing
doc_name_list = []
doc_train_list = []
doc_test_list = []

f = open('data/graph_node_labels.txt', 'r')
lines = f.readlines()

# Process only the lines with the given indices
for i, line in enumerate(lines):
    if i in node_ids:
        doc_name_list.append(line.strip())
        temp = line.split("\t")
        if temp[1].find('test') != -1:
            doc_test_list.append(line.strip())
        elif temp[1].find('train') != -1:
            doc_train_list.append(line.strip())

f.close()
print(doc_train_list)
print(doc_test_list)

['277\ttrain\t16', '2940\ttrain\t24', '4523\ttrain\t30', '8875\ttrain\t24', '10707\ttrain\t28', '11692\ttrain\t4', '12662\ttrain\t24', '15854\ttrain\t8', '16625\ttrain\t16', '17237\ttrain\t16', '20589\ttrain\t27', '23076\ttrain\t28', '27948\ttrain\t10', '29036\ttrain\t28', '29235\ttrain\t5', '29759\ttrain\t4', '31451\ttrain\t23', '32412\ttrain\t39', '32814\ttrain\t2', '33882\ttrain\t2', '34055\ttrain\t34', '36239\ttrain\t25', '41936\ttrain\t20', '44082\ttrain\t32', '44377\ttrain\t16', '45303\ttrain\t4', '47025\ttrain\t34', '50327\ttrain\t24', '53849\ttrain\t28', '55852\ttrain\t24', '57309\ttrain\t31', '61450\ttrain\t30', '62285\ttrain\t25', '62326\ttrain\t28', '69260\ttrain\t19', '72672\ttrain\t22', '75064\ttrain\t16', '78367\ttrain\t26', '80502\ttrain\t24', '81254\ttrain\t27', '81835\ttrain\t11', '82143\ttrain\t28', '83408\ttrain\t28', '84070\ttrain\t16', '85138\ttrain\t19', '90013\ttrain\t34', '90677\ttrain\t4', '91369\ttrain\t31', '92874\ttrain\t4', '94916\ttrain\t24', '95232\ttrain

In [6]:

doc_content_list = []
f = open('data/corpus/clean_100.txt', 'r')
lines = f.readlines()
for line in lines:
    doc_content_list.append(line.strip())
f.close()
print(doc_content_list[:5])


['construction q constant weight sequences using like approach present scheme constant weight sequences , , given information sequence , construction results sequence specific weight within certain scheme uses design based codes , weight sequences , applications', 'decision study agent population identify supported proposal change , space deliberation process agents , two new proposal , agents identify class spaces , spaces , deliberation process result success deliberation , long supported proposal consider general proposal space deliberation setting study conditions deliberation success analysis dynamic algorithm identify supported proposal', 'decentralized dynamic optimization power network voltage control voltage control power distribution networks devices devices also provide limited power resources used network wide voltage decentralized voltage control strategy voltage mismatch error using gradient \\( \\) power network , local voltage provide gradient information paper aims ana

In [7]:
doc_content_list = []
f = open('data/corpus/clean_100.txt', 'r')
lines = f.readlines()
for line in lines:
    doc_content_list.append(line.strip())
f.close()
# print(doc_content_list)

train_ids = []
for train_name in doc_train_list:
    train_id = doc_name_list.index(train_name)
    train_ids.append(train_id)
print(train_ids)
random.shuffle(train_ids)

# partial labeled data
#train_ids = train_ids[:int(0.2 * len(train_ids))]

train_ids_str = '\n'.join(str(index) for index in train_ids)
f = open('data/train_100.index', 'w')
f.write(train_ids_str)
f.close()

test_ids = []
for test_name in doc_test_list:
    test_id = doc_name_list.index(test_name)
    test_ids.append(test_id)
print(test_ids)
random.shuffle(test_ids)

test_ids_str = '\n'.join(str(index) for index in test_ids)
f = open('data/test_100.index', 'w')
f.write(test_ids_str)
f.close()

ids = train_ids + test_ids
print(ids)
print(len(ids))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[]
[46, 27, 12, 35, 10, 94, 50, 88, 7, 72, 18, 95, 16, 26, 55, 97, 70, 11, 91, 51, 3, 36, 2, 75, 1, 78, 30, 89, 76, 53, 63, 0, 17, 44, 38, 74, 52, 32, 34, 21, 6, 71, 82, 62, 9, 33, 99, 80, 81, 24, 37, 86, 69, 65, 42, 64, 48, 22, 61, 14, 20, 39, 66, 84, 23, 8, 4, 85, 31, 13, 90, 5, 57, 58, 92, 40, 54, 98, 60, 45, 87, 25, 73, 19, 67, 49, 29, 41, 93, 47, 68, 83, 56, 15, 77, 28, 43, 79, 59, 96]
100


In [8]:
shuffle_doc_name_list = []
shuffle_doc_words_list = []
for id in ids:
    shuffle_doc_name_list.append(doc_name_list[int(id)])
    shuffle_doc_words_list.append(doc_content_list[int(id)])
shuffle_doc_name_str = '\n'.join(shuffle_doc_name_list)
shuffle_doc_words_str = '\n'.join(shuffle_doc_words_list)

f = open('data/shuffle_100.txt', 'w')
f.write(shuffle_doc_name_str)
f.close()

f = open('data/corpus/shuffle_100.txt', 'w')
f.write(shuffle_doc_words_str)
f.close()
print(shuffle_doc_name_list)

['90677\ttrain\t4', '50327\ttrain\t24', '27948\ttrain\t10', '72672\ttrain\t22', '20589\ttrain\t27', '162540\ttrain\t19', '95232\ttrain\t24', '157799\ttrain\t7', '15854\ttrain\t8', '140989\ttrain\t24', '32814\ttrain\t2', '163206\ttrain\t24', '31451\ttrain\t23', '47025\ttrain\t34', '110223\ttrain\t10', '163884\ttrain\t16', '136795\ttrain\t16', '23076\ttrain\t28', '161232\ttrain\t16', '96890\ttrain\t5', '8875\ttrain\t24', '75064\ttrain\t16', '4523\ttrain\t30', '143573\ttrain\t31', '2940\ttrain\t24', '145422\ttrain\t24', '57309\ttrain\t31', '158097\ttrain\t16', '143857\ttrain\t24', '100687\ttrain\t18', '117732\ttrain\t24', '277\ttrain\t16', '32412\ttrain\t39', '85138\ttrain\t19', '80502\ttrain\t24', '142479\ttrain\t33', '98216\ttrain\t39', '62285\ttrain\t25', '69260\ttrain\t19', '36239\ttrain\t25', '12662\ttrain\t24', '139893\ttrain\t8', '150656\ttrain\t5', '117613\ttrain\t16', '17237\ttrain\t16', '62326\ttrain\t28', '167093\ttrain\t28', '148029\ttrain\t16', '149468\ttrain\t30', '44377\ttr

In [9]:

# build vocab
word_freq = {}
word_set = set()
for doc_words in shuffle_doc_words_list:
    words = doc_words.split()
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)
vocab

['studied',
 'distortion',
 'generalization',
 'must',
 'query',
 'perspective',
 'continuous',
 'application',
 'loss',
 'issue',
 'synthesized',
 'layer',
 'explicitly',
 'recommendation',
 'conventional',
 'technique',
 'privacy',
 'filter',
 'comprehension',
 '0',
 'sample',
 'short',
 'eavesdropper',
 'literature',
 'ratio',
 'specific',
 'end',
 'communication',
 'actually',
 'rates',
 'like',
 'computing',
 'top',
 'computer',
 'weak',
 'inference',
 'finally',
 'binary',
 'synthetic',
 'users',
 'large',
 'occlusion',
 'compare',
 'reduction',
 'single',
 'test',
 'recently',
 'weight',
 'estimation',
 'parameters',
 'parts',
 'independent',
 'modeling',
 'problems',
 'transmitter',
 'mismatch',
 'proof',
 'proposal',
 '\\)',
 'allowing',
 'recurrent',
 'vectors',
 'constraints',
 'state',
 'electricity',
 'benefits',
 'hard',
 'practice',
 'emotion',
 'basis',
 'speech',
 'give',
 'complexity',
 'cvae',
 'extension',
 'defined',
 'supported',
 'coding',
 'dynamic',
 'well',
 '

In [10]:

word_doc_list = {}

for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    appeared = set()
    for word in words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)

word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

vocab_str = '\n'.join(vocab)

f = open('data/corpus/vocab_100.txt', 'w')
f.write(vocab_str)
f.close()

In [11]:
# label list
label_set = set()
for doc_meta in shuffle_doc_name_list:
    temp = doc_meta.split('\t')
    label_set.add(temp[2])
label_list = list(label_set)

label_list_str = '\n'.join(label_list)
f = open('data/corpus/labels_100.txt', 'w')
f.write(label_list_str)
f.close()

In [12]:
train_size = len(train_ids)
val_size = int(0.1 * train_size)
real_train_size = train_size - val_size  # - int(0.5 * train_size)
# different training rates

real_train_doc_names = shuffle_doc_name_list[:real_train_size]
real_train_doc_names_str = '\n'.join(real_train_doc_names)

f = open('data/real_train_100.name', 'w')
f.write(real_train_doc_names_str)
f.close()

In [13]:

row_x = []
col_x = []
data_x = []
for i in range(real_train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            # print(doc_vec)
            # print(np.array(word_vector))
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_x.append(i)
        col_x.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_x.append(doc_vec[j] / doc_len)  # doc_vec[j]/ doc_len

# x = sp.csr_matrix((real_train_size, word_embeddings_dim), dtype=np.float32)
x = sp.csr_matrix((data_x, (row_x, col_x)), shape=(
    real_train_size, word_embeddings_dim))

y = []
for i in range(real_train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    y.append(one_hot)
y = np.array(y)
print(y)

# tx: feature vectors of test docs, no initial features
test_size = len(test_ids)

row_tx = []
col_tx = []
data_tx = []
for i in range(test_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i + train_size]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_tx.append(i)
        col_tx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_tx.append(doc_vec[j] / doc_len)  # doc_vec[j] / doc_len

# tx = sp.csr_matrix((test_size, word_embeddings_dim), dtype=np.float32)
tx = sp.csr_matrix((data_tx, (row_tx, col_tx)),
                   shape=(test_size, word_embeddings_dim))

ty = []
for i in range(test_size):
    doc_meta = shuffle_doc_name_list[i + train_size]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ty.append(one_hot)
ty = np.array(ty)
print(ty)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
[]


In [14]:
word_vectors = np.random.uniform(-0.01, 0.01,
                                 (vocab_size, word_embeddings_dim))

for i in range(len(vocab)):
    word = vocab[i]
    if word in word_vector_map:
        vector = word_vector_map[word]
        word_vectors[i] = vector

row_allx = []
col_allx = []
data_allx = []

for i in range(train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_allx.append(int(i))
        col_allx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_allx.append(doc_vec[j] / doc_len)  # doc_vec[j]/doc_len
for i in range(vocab_size):
    for j in range(word_embeddings_dim):
        row_allx.append(int(i + train_size))
        col_allx.append(j)
        data_allx.append(word_vectors.item((i, j)))


row_allx = np.array(row_allx)
col_allx = np.array(col_allx)
data_allx = np.array(data_allx)

allx = sp.csr_matrix(
    (data_allx, (row_allx, col_allx)), shape=(train_size + vocab_size, word_embeddings_dim))

ally = []
for i in range(train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ally.append(one_hot)

for i in range(vocab_size):
    one_hot = [0 for l in range(len(label_list))]
    ally.append(one_hot)

ally = np.array(ally)

print(x.shape, y.shape, tx.shape, ty.shape, allx.shape, ally.shape)



(90, 300) (90, 27) (0, 300) (0,) (713, 300) (713, 27)


In [15]:

# allx: the the feature vectors of both labeled and unlabeled training instances
# (a superset of x)
# unlabeled training instances -> words

'''
Doc word heterogeneous graph
'''

# word co-occurence with context windows
window_size = 20
windows = []

for doc_words in shuffle_doc_words_list:
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)


word_window_freq = {}
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])

word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i]
            word_i_id = word_id_map[word_i]
            word_j = window[j]
            word_j_id = word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1

row = []
col = []
weight = []

# pmi as weights

num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    row.append(train_size + i)
    col.append(train_size + j)
    weight.append(pmi)

In [16]:




# word vector cosine similarity as weights

'''
for i in range(vocab_size):
    for j in range(vocab_size):
        if vocab[i] in word_vector_map and vocab[j] in word_vector_map:
            vector_i = np.array(word_vector_map[vocab[i]])
            vector_j = np.array(word_vector_map[vocab[j]])
            similarity = 1.0 - cosine(vector_i, vector_j)
            if similarity > 0.9:
                print(vocab[i], vocab[j], similarity)
                row.append(train_size + i)
                col.append(train_size + j)
                weight.append(similarity)
'''
# doc word frequency
doc_word_freq = {}

for doc_id in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[doc_id]
    words = doc_words.split()
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1

for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        if i < train_size:
            row.append(i)
        else:
            row.append(i + vocab_size)
        col.append(train_size + j)
        idf = log(1.0 * len(shuffle_doc_words_list) /
                  word_doc_freq[vocab[j]])
        weight.append(freq * idf)
        doc_word_set.add(word)

node_size = train_size + vocab_size + test_size
adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))


In [17]:

# dump objects
f = open("data/ind.x", 'wb')
pkl.dump(x, f)
f.close()

f = open("data/ind.y", 'wb')
pkl.dump(y, f)
f.close()

f = open("data/ind.tx", 'wb')
pkl.dump(tx, f)
f.close()

f = open("data/ind.ty", 'wb')
pkl.dump(ty, f)
f.close()

f = open("data/ind.allx", 'wb')
pkl.dump(allx, f)
f.close()

f = open("data/ind.ally", 'wb')
pkl.dump(ally, f)
f.close()

f = open("data/ind.adj", 'wb')
pkl.dump(adj, f)
f.close()